# CP421 Project - Recommendation System(CF)
#### Shawn Davis
#### Adam Cassidy
#### Mengdan Wan 

##### Import rating and movie data

In [74]:
import pandas as pd
import numpy as np

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv', skiprows=1, sep=',', header=None, names=rnames)

ratings[:]


,user_id,movie_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [75]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-latest-small/ml-latest-small/movies.csv', skiprows=1, sep=',', header=None, names=mnames,)

movies

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


##### Merge movies and ratings

In [76]:
data = pd.merge(ratings, movies)
data

,user_id,movie_id,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


##### Random split data for training and test

In [77]:
data_copy = data.copy()
train_set = data_copy.sample(frac=0.9) #random choose sample
test_set = data_copy.drop(train_set.index)

train_set[:]

,user_id,movie_id,rating,timestamp,title,genres
44242,201,1876,4.0,941211417,Deep Impact (1998),Drama|Sci-Fi|Thriller
61135,599,2171,2.5,1498516482,Next Stop Wonderland (1998),Comedy|Drama|Romance
45301,600,784,2.5,1237851321,"Cable Guy, The (1996)",Comedy|Thriller
47675,495,89745,4.0,1458635395,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
25202,304,36,1.0,881428195,Dead Man Walking (1995),Crime|Drama
...,...,...,...,...,...,...
25359,70,150,4.0,1355198045,Apollo 13 (1995),Adventure|Drama|IMAX
39461,41,34319,3.0,1458943268,"Island, The (2005)",Action|Sci-Fi|Thriller
22482,390,2076,4.5,1250339036,Blue Velvet (1986),Drama|Mystery|Thriller
6831,160,1196,4.0,971113406,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi


In [78]:
train_set.describe()

,user_id,movie_id,rating,timestamp
count,90752.000000,90752.000000,90752.000000,9.075200e+04
mean,326.173054,19379.651159,3.499879,1.205700e+09
std,182.578896,35455.112451,1.042652,2.161923e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.016842e+09
50%,325.000000,2990.000000,3.500000,1.186086e+09
75%,477.000000,8033.000000,4.000000,1.435993e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


##### Get mean rating of movies

In [79]:
ratings = pd.DataFrame(train_set.groupby('title')['rating'].mean())
ratings

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
...,...
[REC]³ 3 Génesis (2012),3.000000
eXistenZ (1999),3.875000
xXx (2002),2.770833


##### Count how many ratings of each movie

In [80]:
ratings['number_of_ratings'] = data.groupby('title')['rating'].count()
ratings

,rating,number_of_ratings
title,,
'71 (2014),4.000000,1
'Hellboy': The Seeds of Creation (2004),4.000000,1
'Round Midnight (1986),3.500000,2
'Salem's Lot (2004),5.000000,1
'Til There Was You (1997),4.000000,2
...,...,...
[REC]³ 3 Génesis (2012),3.000000,2
eXistenZ (1999),3.875000,22
xXx (2002),2.770833,24


In [81]:
#import seaborn as sns
#sns.jointplot(x='rating', y='number_of_ratings', data=ratings)

In [82]:
train_matrix = pd.pivot_table(train_set, values='rating', index='user_id', columns='title')
#train_matrix.fillna(0, inplace=True)
train_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN


##### Get pearson correlation matrix of movies

In [83]:
corr_matrix = train_matrix.corr(method = 'pearson')
corr_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[REC]³ 3 Génesis (2012),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,-0.164845,NaN,-0.388883,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.762493,NaN,-1.0
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.831858,NaN,...,0.184637,NaN,NaN,-0.086066,-1.0,NaN,0.762493,1.000000,NaN,0.5


In [84]:
#test_matrix = pd.pivot_table(test_set, values='rating', index='user_id', columns='title')
#test_matrix

##### Read test data and convert it to numpy array

In [85]:
test_set.drop(['movie_id', 'timestamp', 'genres'], axis=1, inplace=True)
test_set

,user_id,rating,title
6,19,4.0,Toy Story (1995)
14,44,3.0,Toy Story (1995)
15,45,4.0,Toy Story (1995)
41,124,4.0,Toy Story (1995)
69,193,2.0,Toy Story (1995)
...,...,...,...
100789,610,3.5,13 Sins (2014)
100800,610,3.5,Tusk (2014)
100814,610,1.5,The Gallows (2015)
100817,610,3.0,Goodnight Mommy (Ich seh ich seh) (2014)


In [86]:
print(len(test_set))
test_array = test_set.to_numpy()
test_array

10084


array([[19, 4.0, 'Toy Story (1995)'],
       [44, 3.0, 'Toy Story (1995)'],
       [45, 4.0, 'Toy Story (1995)'],
       ...,
       [610, 1.5, 'The Gallows (2015)'],
       [610, 3.0, 'Goodnight Mommy (Ich seh ich seh) (2014)'],
       [610, 4.5, 'Sympathy for the Underdog (1971)']], dtype=object)

##### Go through every test data, and using pearson correlation matrix to find top k similar movies that already rated by current user. 
##### Then make prediction of rating.

In [91]:
k = 20
c=0
noOfRatingThreshold = 20
for m in test_array:
    t = m[2]
    u = m[0]
    if (corr_matrix.index == t).any():
        movie_corr = pd.DataFrame(corr_matrix[t])
        movie_corr.dropna(inplace=True)
        #movie_corr = movie_corr.join(ratings['number_of_ratings'])
        #movie_corr = movie_corr[movie_corr['number_of_ratings'] > noOfRatingThreshold]
        movie_corr = movie_corr[movie_corr[t] < 1] 
        movie_corr = movie_corr[movie_corr[t] > 0] 
        sim_movies = movie_corr.sort_values(by=t, ascending=False)
        #sim_movies
        sim_title = sim_movies.index
        #print(sim_title)

        count = 0
        r_s = 0
        total_sim = 0
        for movie in sim_title:
            r = train_matrix.loc[u, movie]
            #print(u)
            #print(movie)
            if not np.isnan(r):
                #print(r)
                count += 1
                sim = sim_movies.loc[movie, t]
                r_s += r*sim
                total_sim += sim
            if count >= k:
                break
                #print(count)
        if total_sim == 0:
            print("No Commendation!(No similar movies)")
        else:
            rate = r_s/total_sim
            print(rate)
    else:
        print("No Recommendation(New movie)")
    c += 1
print("Total movies: %d" % c)



2.199376749886512
2.9567557712912924
3.5188841270452156
3.7489680681450612
3.664958381070278
4.021791317745285
2.668632833678997
3.719401638024464
4.312728515112875
3.997608746028647
3.2190717311356187
3.957366238618502
2.8249176730758006
3.3740374282103236
3.763590435475746
3.1923702331697306
3.1476587235860345
4.75922161256554
3.4041511208608513
3.9681281988320922
2.4022893319165504
3.473308144326054
3.7524065502591615
3.739922072823353
3.2851387685774087
3.4130341971798055
4.178643131111394
3.439614674978982
3.548509299291903
3.3747861983637555
4.207374018413111
3.8509605942948637
4.774898904585006
3.754310339950138
4.1841984820521185
3.843211537676469
3.297496085099461
3.658408476161616
3.0637608178369478
4.157989195232345
3.3005215819946185
3.544761535452057
3.3884963139649833
3.7704098829277157
3.547345319540304
3.538481224754551
3.5540150442185383
4.411975873286403
3.9734238536619384
3.0596468716920833
4.277481078666123
4.464884321285999
3.1516626982362324
3.2402900470792737
3.6

2.6019094683437154
2.704443217112389
4.223234063327679
2.6002314390372665
4.229730595458311
3.1817767164362976
3.609841772562447
3.0818424344791766
3.4161053248981106
3.641544410799882
3.3538065541394073
3.4590124057593834
4.165000337565099
4.349678579174731
3.7821727625134627
2.6307365819709836
3.0953924746683956
3.4076202348371165
2.124279062216961
3.415702129282743
3.6212104252602404
3.192003188478101
2.9453232711444475
3.8268167016189665
3.884029480385849
4.014298259480277
3.8180457417109626
3.1235765547977485
3.3247135521639803
2.7010232215968815
4.451568258081867
2.538478332483131
2.6409051661383693
2.517123087431156
3.116834712846326
3.8391426198334
3.4588556974942706
4.725383805039534
3.575148989547502
3.347463649633642
4.442686072414741
3.6028601478901607
3.4298894470650305
3.2572680974182338
3.8379157203851704
4.215154549868618
4.145765519044548
3.5467025954207267
3.116874856213847
3.22313570780687
3.9423719859102504
3.0380327196890984
3.3105142194820303
3.3662913791162814
3.

2.9613976970314346
2.9738377499921516
3.8209327118577745
4.449982195434101
3.3850950254339107
3.3874728798273908
3.698092822185906
3.0647944803602996
2.833415801540795
4.6657759422591765
4.329177208732141
2.475053224260923
3.9909089139113036
4.418328247863921
4.143898736121104
3.534553253368907
3.9712425838146133
3.776836919420531
3.7000432636483924
2.730276938511038
3.8767114673667584
3.640866219180516
3.5798494677754187
4.097423877799743
4.429989476138706
4.0187845419445765
3.4016235927352954
3.3749117248424167
2.7326994383902137
3.0256820220170364
2.917249643405016
3.708859050162245
3.396085167499958
4.587200703471851
3.6239570317793834
3.2303407024701865
3.6160866750462226
3.6603009279185885
3.2535130894066846
4.365638459852433
2.8721198779587125
4.658712421101247
3.936198423442814
3.2470414580311924
4.298119817610957
3.269657009008168
3.892839528300472
4.137937178368335
3.6346723114049553
4.1143869840721266
3.667094721672012
3.519339522191547
4.164426236186574
3.4770663713811825
4

3.71940520767526
3.4950339395560013
3.8078597240393295
4.025362082147844
3.296309750403475
4.447852891835321
2.5773500216417675
2.5118994739958165
3.5212470247990444
3.742041722755617
3.4836786746518005
3.218698731084892
3.432661848909787
3.151419904939006
3.4522352043702482
3.433687066309139
2.4874933188226516
4.24864153803626
3.4257901439023306
4.4820934538169155
3.0073705652624882
3.4832701676048106
3.633376964840872
3.3055686045143355
3.807270241936365
4.443018726134777
3.308653918348659
3.715410730148489
3.4160355984584427
3.4837705189352883
3.1040664396508575
3.8130405818149424
2.412813054812697
3.229594233052276
3.9357235323047113
2.0615412763322896
3.4282472964065573
3.9577587144361885
4.7674653105304845
2.7010885606736164
2.3290816254773503
3.4999482094945167
4.4294568054469075
4.449201675507981
2.8728117768976764
3.23254501823462
4.241356673138485
3.652855048505626
3.6479026578632237
3.9739731332318455
2.4430874180859408
3.495456360980378
3.13167116652243
3.822626242511196
3.

2.5537752555632496
3.6410288820333587
3.434212162643298
3.9933377331205295
3.8340719413979425
4.176957188148073
3.335894624717088
4.392247406932195
3.8229620428677995
2.2813554675913768
2.37496568917855
3.0747801947036684
No Commendation!(No similar movies)
3.473960383216371
3.513970062405388
3.5253333760948875
2.775557827389404
3.469973002330117
4.529654649598014
3.610855991878586
3.5497366768694114
3.725418577309209
3.149494845594199
2.9873590061191915
3.881562040887092
4.518129249051712
3.8777794323041714
3.371293423393182
3.78375778119263
3.372535556956151
3.41291891529651
4.005439313449581
3.9563507644581053
3.5727832599011857
3.5917129832509067
3.134237210513457
3.370423566344386
3.652315409115248
3.4406460128398955
3.949555543522843
4.354344102390723
3.4953787580003586
3.8825615500588295
3.700544729614688
3.898865548939671
2.9468002320169466
3.3736333290786678
3.6298306689992264
3.375512978582071
3.4686822771411085
3.076080465003324
3.2248593949461766
3.041781035100384
2.6624272

...

3.8787047052452297
3.9350414362606347
3.650702968732409
4.514028105839827
4.318018556562073
No Commendation!(No similar movies)
3.963461752861248
4.361336422022964
3.875801063476496
4.450166524891081
3.976304714786369
4.496931836174368
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
4.143099352404711
4.405525996150736
No Commendation!(No similar movies)
No Commendation!(No similar movies)
3.910869299542275
4.494433915713756
4.720811235495224
3.59945413158137
3.244278352500818
2.798232928763178
No Commendation!(No similar movies)
4.104992488130473
2.5392158274984067
3.411375298455219
2.7018252426675495
4.277820962587478
3.000985837068349
3.7083242266833807
3.1769582950423914
5.0
3.344658643972565
3.8536969643722974
4.2698498644385845
2.6798585114052322
4.520197076750054
3.98558802917972
3.376473527214361
3.602255122262467
No Commendation!(No similar movies)
4.70802746091799
3.7055760851502586
3.036295874937049
4.071418631134199

3.1213600820422513
3.0783429358724788
2.5805480639975813
3.6740819152207327
2.969330852585075
2.2972217108231225
3.148570547340435
3.374351046914712
3.199181248085767
2.724684216284641
3.027692738791383
3.2847192101187943
3.8517194183386314
2.656223904992453
3.247581724135425
3.5914208058595696
2.595833369312628
No Commendation!(No similar movies)
3.1923464157674832
No Commendation!(No similar movies)
2.8713315965437274
3.3098319312540094
2.4725551582397314
3.1916457297711798
4.2097851691485415
3.8750285662578268
4.028753950717162
3.564402010387151
3.5212258271304253
2.263023401329884
No Commendation!(No similar movies)
3.3021860617229026
No Commendation!(No similar movies)
3.474117254558999
No Commendation!(No similar movies)
3.2718416422618586
3.7407983013983173
3.2268328209638666
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
2.556440422583731
2.9359840314416616
3.059507049652211
3.8238222035312583
No Commendation!(No simi

3.803310678306211
3.466208477564864
3.4029264100737366
3.1303359289035435
No Commendation!(No similar movies)
No Commendation!(No similar movies)
3.4666346489319766
3.3630029595612347
No Commendation!(No similar movies)
No Commendation!(No similar movies)
3.595388345672572
3.85123069219948
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
3.4230989947570736
3.5692876786455248
3.44404373287693
4.7656231079753395
3.2065108045899304
3.351668906163428
3.4172663849000786
3.1787388141259934
3.8786020722692522
3.076362813947442
4.059631713487595
3.9593476386926953
4.1329529097254945
3.906511723080827
3.7278625235805345
4.347204257164409
4.599292151242277
4.230126557327193
No Commendation!(No similar movies)
No Commendation!(No similar movies)
4.083333333333334
4.096901905016068
No Commendation!(No similar movies)
3.659175281708863
3.5
4.077240809801574
3.54

No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Recommendation(New movie)
3.597526172359533
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
3.0979961649578245
No Commendation!(No similar movies)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Commendation!(No similar movies)
3.9576494594903178
No Commendation!(No similar movies)
3.9103323196301023
3.280770622528215
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No

No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Recommendation(New movie)
No Commendation!(No similar movies)
No Recommendation(New movie)
No Recommendatio